In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate
import json
%config Completer.use_jedi = False  # Fixes autocomplete issues
%config InlineBackend.figure_format = 'retina'  # Improves plot resolution

import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
common_resampled_rate = 1000 #in Hz
event_name = "Apply halt: 2s" #event to align data to, can make into list in the future?

rawdata_paths = [
    Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2721-2024-12-10T14-18-54'),
    Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2722-2024-12-10T14-58-52')
]

rawdata_path = rawdata_paths[0]
data_path = rawdata_path.parent / f"{rawdata_path.name}_processedData/downsampled_data"
save_path = rawdata_path.parent / f"{rawdata_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])

photometry_tracking_encoder_data = pd.read_parquet(data_path / "photometry_tracking_encoder_data.parquet", engine="pyarrow")
camera_photodiode_data = pd.read_parquet(data_path / "camera_photodiode_data.parquet", engine="pyarrow")
experiment_events = pd.read_parquet(data_path / "experiment_events.parquet", engine="pyarrow")
photometry_info = pd.read_parquet(data_path / "photometry_info.parquet", engine="pyarrow")
session_settings = pd.read_parquet(data_path / "session_settings.parquet", engine="pyarrow")
session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)

print(f"✅ Finished loading all parquet files")

In [ ]:
#---------------------------------------------------
# PLOT FIGURE to ascertain everything is well loaded
#---------------------------------------------------

df_to_analyze = photometry_tracking_encoder_data["Photodiode_int"] #using downsampled values in common time grid 
#df_to_analyze = camera_photodiode_data["Photodiode"] #use async raw values if needed for troubleshooting, but the nearest indices needs to be found , see couple of lines below
photodiode_halts, photodiode_delay_min, photodiode_delay_avg, photodiode_delay_max = process.analyze_photodiode(df_to_analyze, experiment_events, event_name, plot = True)
# nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
# photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices]
process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, photodiode_halts, save_figure = False, show_figure = True, downsample_factor=50)